In [35]:
import json
import os
import cv2
import numpy as np

In [36]:
in_dir = "/mnt/nis_lab_research/data/coco_files/neg/shah_b1_539_21_neg"
out_dir = "./output"

In [37]:
def createDataDict (fn):
    
    master_list = []

    with open(fn, "r") as f:
        json_obj = json.load(f)
        
    
        
    for img in json_obj["images"]:
        
        fn = os.path.join(in_dir, "images", img["file_name"][2:])
        h = img["height"]
        w = img["width"]
        ann_list = []
        
        for ann in json_obj["annotations"]:
            if ann["image_id"] == img["id"]:
                ann_list.append(ann)
    
        data_dict = {
            "file_name": fn,
            "height": h,
            "width": w, 
            "annotations": ann_list
        }
 
        master_list.append(data_dict)
        
    return master_list

In [38]:
data_dict_list = createDataDict(os.path.join(in_dir, "result.json"))

In [39]:
# Create a dictionary to map category_ids to unique colors
category_colors = {}

# Get unique category_ids
unique_category_ids = set()
for data_dict in data_dict_list:
    for annotation in data_dict['annotations']:
        unique_category_ids.add(annotation['category_id'])

# Generate unique colors for each category
for category_id in unique_category_ids:
    color = tuple(np.random.randint(0, 256, 3).tolist())  # Generate a random color for each category
    category_colors[category_id] = color

In [40]:
# Define a dictionary to map category_ids to labels (customize as needed)

# # Option 1
# category_labels = {
#     0: 'Accept Button',
#     1: 'Address Input Box',
#     2: 'Advertisement',
#     3: 'Alert Notification',
#     4: 'Allow Button',
#     5: 'Checkbox',
#     6: 'Click Captcha',
#     7: 'Close Button',
#     8: 'Download Button',
#     9: 'Email Input Box',
#     10: 'General Button',
#     11: 'General Input Box',
#     12: 'Image Captcha',
#     13: 'Login Button',
#     14: 'Logo',
#     15: 'Name Input Box',
#     16: 'Password Input Box',
#     17: 'Phone Input Box',
#     18: 'Play Button',
#     19: 'Popup',
#     20: 'Search Button',
#     21: 'Search Input Box',
#     22: 'Submit Button',
#     23: 'Text Captcha',
#     24: 'Toggle Button',
#     25: 'Update Button',
#     26: 'Video'
# }

# Option 2
category_labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9',
    10: '10',
    11: '11',
    12: '12',
    13: '13',
    14: '14',
    15: '15',
    16: '16',
    17: '17',
    18: '18',
    19: '19',
    20: '20',
    21: '21',
    22: '22',
    23: '23',
    24: '24',
    25: '25',
    26: '26',
    27: '27'
}

In [41]:
# Create the output directory if it doesn't exist
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [42]:
# Loop through data_dict_list and draw bounding boxes with category labels
for data_dict in data_dict_list:
    image_path = data_dict['file_name']
    image = cv2.imread(image_path)

    for annotation in data_dict['annotations']:
        bbox = annotation['bbox']
        x, y, w, h = bbox
        x, y, w, h = int(x), int(y), int(w), int(h)

        category_id = annotation['category_id']
        category_label = category_labels.get(category_id, 'Unknown')
        category_color = category_colors.get(category_id, (0, 0, 255))  # Default to blue if not in mapping

        cv2.rectangle(image, (x, y), (x + w, y + h), category_color, 2)  # Draw the bounding box
        cv2.putText(image, category_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, category_color, 2)

    # Save the image with bounding boxes and labels to the output directory
    output_path = os.path.join(out_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, image)